# Accessing storage in Databricks
---
## 0. Prerequisites
### 0.1. Azure App registration

1. [Create an Azure AD application and service principal that can access resources](https://docs.microsoft.com/en-us/azure/azure-resource-manager/resource-group-create-service-principal-portal). Note the following properties:
  - application-id: An ID that uniquely identifies the application
  - directory-id: An ID that uniquely identifies the Azure AD instance
  - storage-account-name: The name of the storage account
  - service-credential: A string that the application uses to prove its identity   
  
2. Register the service principal, granting the correct role assignment, such as Storage Blob Data Contributor, on the Azure Data Lake Storage Gen2 account.

### 0.2. Adding scoped secrets
To add a secret and a scope, this needs to be completed using the [Databricks CLI](https://docs.databricks.com/dev-tools/cli/index.html).

Once signed in via the CLI, issue the following commands:
1. `databricks secrets create-scope --scope Analysts`
2. `databricks secrets put --scope Analysts --key SPID --string-value "Service Principal ID"` (Application Client ID)
3. `databricks secrets put --scope Analysts --key SPKey --string-value "Service Principal Secret Key"`
4. `databricks secrets put --scope Analysts --key DirectoryID --string-value "Azure Directory ID"`

> Update: This can also be done via the browser via the URL **https://<\location>.azuredatabricks.net/?o=<\orgID>#secrets/createScope**

## 1. Mount Azure Data Lake Storage Gen2 filesystem
This only needs to be done once per cluster

In [4]:
try:
    dbutils.fs.unmount('/mnt/ecovacs')
except:
    print()
    
try:
    dbutils.fs.unmount('/mnt/transactions')
except:
    print()

/mnt/ecovacs has been unmounted.

In [5]:
# service principal details
client_id     = dbutils.secrets.get(scope='Analysts', key='SPID')
client_secret = dbutils.secrets.get(scope='Analysts', key='SPKey')
directory_id  = dbutils.secrets.get(scope='Analysts', key='DirectoryID')

# data lake container details
account_name = 'storagejamesleslie'
container_name = 'ecovacs'

configs = {'fs.azure.account.auth.type': 'OAuth',
           'fs.azure.account.oauth.provider.type': 'org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider',
           'fs.azure.account.oauth2.client.id': client_id,
           'fs.azure.account.oauth2.client.secret': client_secret,
           'fs.azure.account.oauth2.client.endpoint': f'https://login.microsoftonline.com/{directory_id}/oauth2/token'}

# Optionally, you can add <directory-name> to the source URI of your mount point.
dbutils.fs.mount(
    source = f'abfss://{container_name}@{account_name}.dfs.core.windows.net/',
    mount_point = f'/mnt/{container_name}',
    extra_configs = configs)

# list all items in container, use display() for nicer formatting
display(dbutils.fs.ls(f'/mnt/{container_name}'))

path,name,size
dbfs:/mnt/ecovacs/BestBuy/,BestBuy/,0
dbfs:/mnt/ecovacs/VSI/,VSI/,0


### Unmount a mount point

In [7]:
# unmount container
dbutils.fs.unmount(f'/mnt/{container_name}')

/mnt/ecovacs has been unmounted.
Out[10]: True

## 2. Access directly with service principal and OAuth 2.0
All code below needs to be run every time

In [9]:
# service principal details
client_id     = dbutils.secrets.get(scope='Analysts', key='SPID')
client_secret = dbutils.secrets.get(scope='Analysts', key='SPKey')
directory_id  = dbutils.secrets.get(scope='Analysts', key='DirectoryID')

# update spark configs
spark.conf.set('fs.azure.account.auth.type', 'OAuth')
spark.conf.set('fs.azure.account.oauth.provider.type', 'org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider')
spark.conf.set('fs.azure.account.oauth2.client.id', client_id)
spark.conf.set('fs.azure.account.oauth2.client.secret', client_secret)
spark.conf.set('fs.azure.account.oauth2.client.endpoint', f'https://login.microsoftonline.com/{directory_id}/oauth2/token')

# list all items in container / directory
display(dbutils.fs.ls('abfss://ecovacs@storagejamesleslie.dfs.core.windows.net/BestBuy/'))

path,name,size
abfss://ecovacs@storagejamesleslie.dfs.core.windows.net/BestBuy/archive/,archive/,0
abfss://ecovacs@storagejamesleslie.dfs.core.windows.net/BestBuy/delta/,delta/,0
abfss://ecovacs@storagejamesleslie.dfs.core.windows.net/BestBuy/errors/,errors/,0
abfss://ecovacs@storagejamesleslie.dfs.core.windows.net/BestBuy/processed/,processed/,0


---
# Using dbutils

In [11]:
dbutils.fs.help()

dbutils.fs provides utilities for working with FileSystems. Most methods in
this package can take either a DBFS path (e.g., "/foo" or "dbfs:/foo"), or
another FileSystem URI.

For more info about a method, use dbutils.fs.help("methodName") .

In notebooks, you can also use the %fs shorthand to access DBFS. The %fs shorthand maps
straightforwardly onto dbutils calls. For example, "%fs head --maxBytes=10000 /file/path"
translates into "dbutils.fs.head("/file/path", maxBytes = 10000)".
 fsutils cp(from: String, to: String, recurse: boolean = false): boolean -> Copies a file or directory, possibly across FileSystems head(file: String, maxBytes: int = 65536): String -> Returns up to the first 'maxBytes' bytes of the given file as a String encoded in UTF-8 ls(dir: String): Seq -> Lists the contents of a directory mkdirs(dir: String): boolean -> Creates the given directory if it does not exist, also creating any necessary parent directories mv(from: String, to: String, recurse: boolean = false): boolean -> Moves a file or directory, possibly across FileSystems put(file: String, contents: String, overwrite: boolean = false): boolean -> Writes the given String out to a file, encoded in UTF-8 rm(dir: String, recurse: boolean = false): boolean -> Removes a file or directory mount mount(source: String, mountPoint: String, encryptionType: String = "", owner: String = null, extraConfigs: Map = Map.empty[String, String]): boolean -> Mounts the given source directory into DBFS at the given mount point mounts: Seq -> Displays information about what is mounted within DBFS refreshMounts: boolean -> Forces all machines in this cluster to refresh their mount cache, ensuring they receive the most recent information unmount(mountPoint: String): boolean -> Deletes a DBFS mount point

## 1. Listing files / directories

In [13]:
# Mount the "transactions" container
dbutils.fs.mount(
    source = f'abfss://transactions@{account_name}.dfs.core.windows.net/',
    mount_point = f'/mnt/transactions',
    extra_configs = configs)

# list all mounted directories
display(dbutils.fs.ls('/mnt'))

path,name,size
dbfs:/mnt/ecovacs/,ecovacs/,0
dbfs:/mnt/transactions/,transactions/,0


In [14]:
# list all items in the 2020 directory
display(dbutils.fs.ls('/mnt/transactions/2020'))

path,name,size
dbfs:/mnt/transactions/2020/p01/,p01/,0
dbfs:/mnt/transactions/2020/p02/,p02/,0
dbfs:/mnt/transactions/2020/p03/,p03/,0
dbfs:/mnt/transactions/2020/p04/,p04/,0
dbfs:/mnt/transactions/2020/p05/,p05/,0
dbfs:/mnt/transactions/2020/p06/,p06/,0
dbfs:/mnt/transactions/2020/p07/,p07/,0
dbfs:/mnt/transactions/2020/p08/,p08/,0
dbfs:/mnt/transactions/2020/p09/,p09/,0
dbfs:/mnt/transactions/2020/p10/,p10/,0


## 2. Creating new directories

In [16]:
# create a new directory
dbutils.fs.mkdirs('/mnt/transactions/2020/p14')

# check that it appears in the mount point
display(dbutils.fs.ls('/mnt/transactions/2020'))

path,name,size
dbfs:/mnt/transactions/2020/p01/,p01/,0
dbfs:/mnt/transactions/2020/p02/,p02/,0
dbfs:/mnt/transactions/2020/p03/,p03/,0
dbfs:/mnt/transactions/2020/p04/,p04/,0
dbfs:/mnt/transactions/2020/p05/,p05/,0
dbfs:/mnt/transactions/2020/p06/,p06/,0
dbfs:/mnt/transactions/2020/p07/,p07/,0
dbfs:/mnt/transactions/2020/p08/,p08/,0
dbfs:/mnt/transactions/2020/p09/,p09/,0
dbfs:/mnt/transactions/2020/p10/,p10/,0


In [17]:
# does it appear in the data lake too?
display(dbutils.fs.ls('abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/'))

path,name,size
abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p01/,p01/,0
abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p02/,p02/,0
abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p03/,p03/,0
abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p04/,p04/,0
abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p05/,p05/,0
abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p06/,p06/,0
abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p07/,p07/,0
abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p08/,p08/,0
abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p09/,p09/,0
abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p10/,p10/,0


## 3. Remove directory

In [19]:
# remove the new directory and all its contents
dbutils.fs.rm('/mnt/transactions/2020/p14', recurse=True)

# has it been removed from the data laker too?
display(dbutils.fs.ls('abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/'))

path,name,size
abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p01/,p01/,0
abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p02/,p02/,0
abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p03/,p03/,0
abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p04/,p04/,0
abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p05/,p05/,0
abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p06/,p06/,0
abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p07/,p07/,0
abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p08/,p08/,0
abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p09/,p09/,0
abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p10/,p10/,0


## Query a SQL Server Database

In [21]:
jdbcHostname = '<server name>.database.windows.net'
jdbcDatabase = '<database name>'
userName = '<user name>'
password = '<password>'  # store this in a Databricks secret scope
jdbcPort = 1433
jdbcUrl = f'jdbc:sqlserver://{jdbcHostname}:{jdbcPort};database={jdbcDatabase};user={userName};password={password}'

df = spark.read.jdbc(url=jdbcUrl, table='<table name>')
display(df)